In [13]:
import sys
import os
print(sys.version)

3.6.10 |Anaconda, Inc.| (default, May  8 2020, 02:54:21) 
[GCC 7.3.0]


In [14]:
import random
import sys
import datetime
import cv2
import numpy as np
import operator
import imutils
#import matplotlib.pyplot as plt

#debug info OpenCV version
print ("OpenCV version: " + cv2.__version__)

OpenCV version: 4.2.0


In [15]:
def display_image(img, title):
    cv2.imshow(title,img) 
    # here it should be the pause ,0 means indefinitely
    k = cv2.waitKey(0)
    if k == 27:         # wait for ESC key to exit
        cv2.destroyAllWindows()

In [16]:
#image path and valid extensions
imagePath = "dataset/test.jpg" #specify your path here
img = cv2.imread(imagePath)

print('Original Dimensions : ',img.shape)
# percent of original size
scale_percent = 10
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
print('Resized Dimensions : ',img.shape)

#Grayscale
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

display_image(img,"GrayResized")

Original Dimensions :  (3264, 2448, 3)
Resized Dimensions :  (326, 244, 3)


In [17]:
#Processing the image
img = cv2.GaussianBlur(img, (9 ,9), 0)
display_image(img,"GausianBlur")

In [18]:
# Adaptive threshold using 11 nearest neighbour pixels
# If the pixel value is smaller than the threshold, it is set to 0, otherwise it is set to a maximum value.
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
display_image(img,"AdaptiveThreshold")

In [19]:
# Invert colours, so gridlines have non-zero pixel values.
img = cv2.bitwise_not(img);
display_image(img,"BitwiseNot")

In [20]:
#increases the white region in the image or size of foreground object increases.This makes gridlines prominent
kernel = np.array([[0., 1., 0.], [1., 1., 1.], [0., 1., 0.]], np.uint8)
img = cv2.dilate(img, kernel)
display_image(img,"Dilated")

In [21]:
def getCornerPoints(img):
    # locate the corners of the sudoku board
    # Find contours
    # Contours is a Python list of all the contours in the image. 
    #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
    contours=cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    # Sort by area, descending
    cnts = imutils.grab_contours(contours)
    c = max(cnts, key=cv2.contourArea)

    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    cv2.drawContours(img, [c], -1, (255, 0, 0), 5)

    img = cv2.circle(img, extLeft, 8, (255, 255, 255), -1)
    img = cv2.circle(img, extRight, 8, (255, 255, 255), -1)
    img = cv2.circle(img, extTop, 8, (255, 255, 255), -1)
    img = cv2.circle(img, extBot, 8, (255, 255,255), -1)

    display_image(img,"Contours")


    return [extLeft,extRight,extTop,extBot]

In [22]:
print(getCornerPoints(img))
def distance_between(p1, p2):
    """Returns the scalar distance between two points"""
    a = p2[0] - p1[0]
    b = p2[1] - p1[1]
    return np.sqrt((a ** 2) + (b ** 2))

[(8, 91), (243, 224), (237, 85), (13, 292)]
